In [1]:
from langchain.llms import GooglePalm
import config
api_key=config.api_key
llm=GooglePalm(google_api_key=api_key,temperature=0.8)


In [2]:
prompt=llm("what is llm")

In [3]:
prompt

'LLLM stands for "language and linguistic methodology". It is a field of study that focuses on the methods used to study language and linguistics.'

In [4]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [5]:
db_user = config.db_user
db_password = config.db_password
db_host = config.db_host
db_name = config.db_name

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [6]:
db_chain = SQLDatabaseChain.from_llm(llm,db,verbose=True)

In [7]:
qns1 = db_chain.run("How many t-shirts do we have left for nike in extra small size and white color?")




> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(94,)]
Answer:94
> Finished chain.


In [8]:
qns2 = db_chain.run("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) 
FROM t_shirts
WHERE size = 'S'
SQLResult: [(Decimal('501'),)]
Answer:501
> Finished chain.


In [9]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('26922'),)]
Answer:26922
> Finished chain.


In [10]:
qns3 = db_chain.run("If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?")




> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(discounted_price) FROM t_shirts AS t LEFT JOIN discounts AS d ON t.t_shirt_id = d.t_shirt_id WHERE d.t_shirt_id IS NOT NULL AND t.brand = 'Levi' AND CURDATE() BETWEEN t.start_date AND t.end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'discounted_price' in 'field list'")
[SQL: SELECT SUM(discounted_price) FROM t_shirts AS t LEFT JOIN discounts AS d ON t.t_shirt_id = d.t_shirt_id WHERE d.t_shirt_id IS NOT NULL AND t.brand = 'Levi' AND CURDATE() BETWEEN t.start_date AND t.end_date]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [11]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)




> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('25547.350000'),)]
Answer:25547.35
> Finished chain.


In [12]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")




> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('27121'),)]
Answer:27121
> Finished chain.


In [13]:
qns5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(40,), (24,)]
Answer:40
> Finished chain.


In [14]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")




> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('64'),)]
Answer:64
> Finished chain.


In [15]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

# Semantic Similarity Based example selector

In [18]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer, util


# model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
# embeddings = f(model_name=model)

embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [19]:
to_vectorize = ["  ".join(example.values()) for example in few_shots]

In [20]:
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color?  SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'  Result of the SQL query  94",
 "How much is the total price of the inventory for all S-size t-shirts?  SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'  Result of the SQL query  26922",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?  SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n   Result of the SQL query  25547.35",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?  SELECT SUM(price * stock_quantity) FROM t_shirts WHERE bran

In [22]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [23]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '94',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '27121',
  'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query'}]

In [24]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""


In [25]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


# PromptTemplete

In [26]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)


In [27]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"],
)

In [28]:
##connecting all to chain

In [29]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)